# Adding two new features

In [2]:
using PyPlot, Compat, DataFrames

In [4]:
dataDirectory = "Student_Team_Competition_Syngenta_Data/";
file = "EXPERIMENT DATA CSV (Cleaned).csv";
readData = dataDirectory * file;
df = readtable(readData);

In [5]:
# Data cleaning

# add new featuers: RM_GRP and YIELD_GRP
df[13] = floor(df[:RM])
df[14] = 10*floor(df[:YIELD]/10)
df


df[15:17] = copy(df[[:CLASS_OF, :GRAD, :BAGSOLD]])
delete!(df, [:CLASS_OF, :GRAD, :BAGSOLD])

names!(df, [:YEAR, :EXPERIMENT, :LOCATION, :VARIETY, :FAMILY, :CHECK, :RM, :REPNO, :YIELD, :RM_GRP, :YIELD_GRP, :CLASS_OF, :GRAD, :BAGSOLD]);

head(df)

,YEAR,EXPERIMENT,LOCATION,VARIETY,FAMILY,CHECK,RM,REPNO,YIELD,RM_GRP,YIELD_GRP,CLASS_OF,GRAD,BAGSOLD
1,2009,09YT000052,3210,V000016,FAM05619,true,3.9,1,50.3023273,3.0,50.0,-1,-1,-1
2,2009,09YT000052,3210,V000016,FAM05619,true,3.9,2,48.3313053,3.0,40.0,-1,-1,-1
3,2009,09YT000052,3210,V000018,FAM05619,true,3.7,1,53.1032456,3.0,50.0,-1,-1,-1
4,2009,09YT000052,3210,V000018,FAM05619,true,3.7,2,53.1032456,3.0,50.0,-1,-1,-1
5,2009,09YT000052,3210,V000039,FAM05625,true,3.4,1,58.0811359,3.0,50.0,-1,-1,-1
6,2009,09YT000052,3210,V000039,FAM05625,true,3.4,2,58.2893163,3.0,50.0,-1,-1,-1


# Grouping avg yield by family and by year

In [6]:
# Split into years 2009 - 2014, excluding class of 2014

yearlist = 2009:2014
dfList = [];
for i in yearlist
    k = copy(df);
    k = k[k[:CLASS_OF].!= 2014,:];
    k = k[k[:YEAR].== i,:];
    push!(dfList,k);
end


In [7]:
# shows the no of data records in each year, excluding class of 2014
for i in 1:6
    @show size(dfList[i])
end

size(dfList[i]) = (11207,14)
size(dfList[i]) = (20240,14)
size(dfList[i]) = (103906,14)
size(dfList[i]) = (82826,14)
size(dfList[i]) = (22029,14)
size(dfList[i]) = (13455,14)


In [8]:
uniquefamilies = []
for year in yearlist 
    groupbyfamily = by(dfList[year-2008], :FAMILY,nrow)
    names!(groupbyfamily, [:FAMILY, :NROWS])
    push!(uniquefamilies,groupbyfamily)
end

for i in 1:6
    @show size(uniquefamilies[i])
end;

size(uniquefamilies[i]) = (244,2)
size(uniquefamilies[i]) = (246,2)
size(uniquefamilies[i]) = (829,2)
size(uniquefamilies[i]) = (871,2)
size(uniquefamilies[i]) = (236,2)
size(uniquefamilies[i]) = (155,2)


In [27]:
function avgYieldPerFamily(i)
    avg_yields =[]
    for k in 1:size(uniquefamilies[i])[1]
        yields = []
        for j in 1:size(dfList[i])[1]

            if (dfList[i][:FAMILY][j] == uniquefamilies[i][1][k] && dfList[i][:YEAR][j] == i+2008)
                push!(yields,dfList[i][:YIELD][j])
            end
           
        end
    push!(avg_yields, mean(yields)) 
    end
   return avg_yields
end ;



In [25]:
    uniquefamilies[1][3] = avgYieldPerFamily(1)
tail(uniquefamilies[1])dddd

,FAMILY,NROWS,x3
1,FAM16708,4,47.748400225
2,FAM16802,4,49.617228049999994
3,FAM16806,5,57.013705859999995
4,FAM16808,6,56.42758888333333
5,FAM16817,6,65.38095603333333
6,FAM16850,17,58.106756064705884


In [28]:
# group average yield by family
for i in 1:6
    uniquefamilies[i][3] = avgYieldPerFamily(i)
    names!(uniquefamilies[i], [:FAMILY, :NROWS, :AVGYIELD])
end

In [29]:
# sort families by avg yield (highest first)
for i in 1:6
    sort!(uniquefamilies[i],cols = order(:AVGYIELD), rev=true)
    #uniquefamilies[i][4] = cumsum(uniquefamilies[i][:NROWS])
  #  names!(uniquefamilies[i], [:FAMILY, :NROWS, :AVGYIELD, :CUM_NROWS])
end

In [35]:
head(uniquefamilies[1])
head(uniquefamilies[3])

,FAMILY,NROWS,AVGYIELD
1,FAM15186,9,68.14601278888887
2,FAM05285,10,67.6675449
3,FAM14334,12,67.62386417500001
4,FAM15536,9,65.7998034111111
5,FAM14139,18,65.68841037222222
6,FAM16815,55,64.8671511090909


In [37]:
# segment the families into 20 percent segments
nrowssegment = []
for i in 1:6
    numrows = size(uniquefamilies[i])[1]
    @show numrows
    twenties = zeros(5)
    for j in 1:5
        twenties[j] = convert(Int64,floor(j*numrows/5))
    end
    push!(nrowssegment, twenties)
end
@show nrowssegment

numrows = 244
numrows = 246
numrows = 829
numrows = 871
numrows = 236
numrows = 155
nrowssegment = Any[[48.0,97.0,146.0,195.0,244.0],[49.0,98.0,147.0,196.0,246.0],[165.0,331.0,497.0,663.0,829.0],[174.0,348.0,522.0,696.0,871.0],[47.0,94.0,141.0,188.0,236.0],[31.0,62.0,93.0,124.0,155.0]]


6-element Array{Any,1}:
 [48.0,97.0,146.0,195.0,244.0]  
 [49.0,98.0,147.0,196.0,246.0]  
 [165.0,331.0,497.0,663.0,829.0]
 [174.0,348.0,522.0,696.0,871.0]
 [47.0,94.0,141.0,188.0,236.0]  
 [31.0,62.0,93.0,124.0,155.0]   

In [38]:
# indicate which 20percent segment family belongs to
for i in 1:6
    uniquefamilies[i][4] = 0 # initialise segment column to 0
    for row in 1:size(uniquefamilies[i])[1] # for every row
        if (row <= nrowssegment[i][1])
            uniquefamilies[i][row,4] = 1
        elseif (row <= nrowssegment[i][2])
            uniquefamilies[i][row, 4] = 2

        elseif (row <= nrowssegment[i][3]) 
            uniquefamilies[i][row, 4] = 3
        elseif (row <= nrowssegment[i][4])
            uniquefamilies[i][row, 4] = 4

        elseif (row <= nrowssegment[i][5])
            uniquefamilies[i][row, 4] = 5

        end
    end
    names!(uniquefamilies[i], [:FAMILY, :NROWS, :AVGYIELD, :SEGMENT])
end
head(uniquefamilies[1])

,FAMILY,NROWS,AVGYIELD,SEGMENT
1,FAM13373,2,73.06856175,1
2,FAM13416,2,72.8101207,1
3,FAM13511,4,72.6569594,1
4,FAM01108,4,72.654096925,1
5,FAM13658,8,71.772601725,1
6,FAM01019,4,71.09806012499999,1


In [39]:
@show numcols = size(df)[2] #14
head(df)

numcols = (size(df))[2] = 14


,YEAR,EXPERIMENT,LOCATION,VARIETY,FAMILY,CHECK,RM,REPNO,YIELD,RM_GRP,YIELD_GRP,CLASS_OF,GRAD,BAGSOLD
1,2009,09YT000052,3210,V000016,FAM05619,true,3.9,1,50.3023273,3.0,50.0,-1,-1,-1
2,2009,09YT000052,3210,V000016,FAM05619,true,3.9,2,48.3313053,3.0,40.0,-1,-1,-1
3,2009,09YT000052,3210,V000018,FAM05619,true,3.7,1,53.1032456,3.0,50.0,-1,-1,-1
4,2009,09YT000052,3210,V000018,FAM05619,true,3.7,2,53.1032456,3.0,50.0,-1,-1,-1
5,2009,09YT000052,3210,V000039,FAM05625,true,3.4,1,58.0811359,3.0,50.0,-1,-1,-1
6,2009,09YT000052,3210,V000039,FAM05625,true,3.4,2,58.2893163,3.0,50.0,-1,-1,-1


In [40]:
# PREPARE for one-hot encoding to indicate the segment of the family's avg yield
for i in 1:size(nrowssegment[1])[1]
    df[numcols+i] = 0
end
numcols = size(df)[2]
df[numcols+1:numcols+3] = copy(df[[:CLASS_OF, :GRAD, :BAGSOLD]])
delete!(df, [:CLASS_OF, :GRAD, :BAGSOLD])
names!(df, [:YEAR, :EXPERIMENT, :LOCATION, :VARIETY, :FAMILY, :CHECK, :RM, :REPNO, :YIELD, :RM_GRP, :YIELD_GRP, :FAM_SEGMENT1, :FAM_SEGMENT2, :FAM_SEGMENT3, :FAM_SEGMENT4, :FAM_SEGMENT5, :CLASS_OF, :GRAD, :BAGSOLD])
head(df)

,YEAR,EXPERIMENT,LOCATION,VARIETY,FAMILY,CHECK,RM,REPNO,YIELD,RM_GRP,YIELD_GRP,FAM_SEGMENT1,FAM_SEGMENT2,FAM_SEGMENT3,FAM_SEGMENT4,FAM_SEGMENT5,CLASS_OF,GRAD,BAGSOLD
1,2009,09YT000052,3210,V000016,FAM05619,true,3.9,1,50.3023273,3.0,50.0,0,0,0,0,0,-1,-1,-1
2,2009,09YT000052,3210,V000016,FAM05619,true,3.9,2,48.3313053,3.0,40.0,0,0,0,0,0,-1,-1,-1
3,2009,09YT000052,3210,V000018,FAM05619,true,3.7,1,53.1032456,3.0,50.0,0,0,0,0,0,-1,-1,-1
4,2009,09YT000052,3210,V000018,FAM05619,true,3.7,2,53.1032456,3.0,50.0,0,0,0,0,0,-1,-1,-1
5,2009,09YT000052,3210,V000039,FAM05625,true,3.4,1,58.0811359,3.0,50.0,0,0,0,0,0,-1,-1,-1
6,2009,09YT000052,3210,V000039,FAM05625,true,3.4,2,58.2893163,3.0,50.0,0,0,0,0,0,-1,-1,-1


In [20]:
function findSegment(i, family)
for row in 1:size(uniquefamilies[i])[1]
    if (uniquefamilies[i][row,:FAMILY] == family)
        return uniquefamilies[i][row, :SEGMENT]    
    end
end
    return -1 
end

findSegment (generic function with 1 method)

In [21]:
# DO one-hot encoding for family segment
for row in 1:size(df)[1]
    for year in 1:6
        segment = findSegment(year, df[row,:FAMILY])
        if (df[row, :YEAR] == year+2008 && segment!=-1)
            if (segment == 1)
                df[row, :FAM_SEGMENT1] = 1
            elseif (segment ==2)
                df[row, :FAM_SEGMENT2] = 1
            elseif (segment ==3)
                df[row, :FAM_SEGMENT3] = 1
            elseif (segment==4)
                df[row, :FAM_SEGMENT4] = 1
            else
                df[row, :FAM_SEGMENT5] = 1
            end

        end
    end
end


# Split data into training, test and prediction set (Class of 2014)

In [22]:
# Split into years 2009 - 2014, excluding class of 2014
# do this again as more columns have been added to df

yearlist = 2009:2014
dfList = [];
for i in yearlist
    k = copy(df);
    k = k[k[:CLASS_OF].!= 2014,:];
    k = k[k[:YEAR].== i,:];
    push!(dfList,k);
end


# training[1] = 2009; test 2010
# training[2] = 2009,2010; test 2011
# training[3] = 2009,2010, 2011; test 2012
# training[4] = 2009,2010,2011,2012; test 2013
# training[5] = 2009,2010,2011,2012,2013; test 2014


#test set
folder_string = "Split_Data/"
extension = ".csv"
for i in yearlist
    writetable(folder_string*"test_"*string(i)*extension,dfList[i-2008])
end


In [23]:
# sizes of each test set
sizes1 = [];
for sets in dfList
    push!(sizes1,size(sets));
end
sizes1

6-element Array{Any,1}:
 (11207,19) 
 (20240,19) 
 (103906,19)
 (82826,19) 
 (22029,19) 
 (13455,19) 

In [24]:
#prediction set
predictionset = copy(df);
predictionset = predictionset[predictionset[:CLASS_OF].== 2014,:];
writetable(folder_string*"prediction_set.csv",predictionset);

#training set
training5 = copy(df);
training5 = training5[training5[:CLASS_OF].!=2014,:];
training5 = training5[training5[:YEAR].!=2014,:];
training4 = copy(training5);
training4 = training4[training4[:YEAR].!=2013,:];
training3 = copy(training4);
training3 = training3[training3[:YEAR].!=2012,:];
training2 = copy(training3);
training2 = training2[training2[:YEAR].!=2011,:];
training1 = copy(dfList[1]);
training = [training1,training2,training3,training4,training5];
for i in 1:length(training)
    writetable(folder_string*"training"* "[" * string(i)  *"]"*extension,training[i]);
end



In [25]:
# sizes of training sets
sizes = [];
for sets in training
    push!(sizes,size(sets));
end
sizes

5-element Array{Any,1}:
 (11207,19) 
 (31447,19) 
 (135353,19)
 (218179,19)
 (240208,19)